In [12]:
import cv2
import os
from tqdm import tqdm

def split_double_eye_images(input_folder, output_folder, threshold_ratio=1.3):
    os.makedirs(output_folder, exist_ok=True)
    skipped = []

    for fname in tqdm(os.listdir(input_folder), desc="Checking for double eyes"):
        try:
            img_path = os.path.join(input_folder, fname)
            img = cv2.imread(img_path)

            if img is None:
                skipped.append((fname, "Unreadable image"))
                continue

            h, w = img.shape[:2]

            # Check aspect ratio: wider than tall likely means 2 eyes
            aspect_ratio = w / h

            if aspect_ratio >= threshold_ratio:
                # Horizontal split
                mid = w // 2
                left_eye = img[:, :mid]
                right_eye = img[:, mid:]

                # Resize both to 224x224
                left_eye = cv2.resize(left_eye, (224, 224))
                right_eye = cv2.resize(right_eye, (224, 224))

                base = os.path.splitext(fname)[0]
                cv2.imwrite(os.path.join(output_folder, f"{base}_left.jpg"), left_eye)
                cv2.imwrite(os.path.join(output_folder, f"{base}_right.jpg"), right_eye)

            else:
                # Just one eye, resize and save
                resized = cv2.resize(img, (224, 224))
                cv2.imwrite(os.path.join(output_folder, fname), resized)

        except Exception as e:
            skipped.append((fname, str(e)))

    return skipped

In [13]:
split_healthy_path = '../data/split/healthy_eye'
split_infected_path = '../data/split/infected_eye'

skipped_h = split_double_eye_images('../data/processed/healthy_eye', split_healthy_path)
skipped_i = split_double_eye_images('../data/processed/infected_eye', split_infected_path)

# Optional: save skipped logs
import pandas as pd
all_skipped = skipped_h + skipped_i
if all_skipped:
    pd.DataFrame(all_skipped, columns=["filename", "reason"]).to_csv("../data/log/skipped_split.csv", index=False)

Checking for double eyes: 100%|██████████| 558/558 [00:00<00:00, 1061.02it/s]


In [14]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

# Folder where original processed images (may contain 2 eyes) are stored
original_folder = '../data/processed/healthy_eye'  # or infected_eye

# Folder where split images are stored
split_folder = '../data/split/healthy_eye'

# Loop over a few original files that were likely split
count = 0
for fname in os.listdir(original_folder):
    base = os.path.splitext(fname)[0]
    left_path = os.path.join(split_folder, f"{base}_left.jpg")
    right_path = os.path.join(split_folder, f"{base}_right.jpg")

    # Check if both splits exist → likely a double-eye image
    if os.path.exists(left_path) and os.path.exists(right_path):
        # Load original and both split crops
        original = cv2.imread(os.path.join(original_folder, fname))
        left = cv2.imread(left_path)
        right = cv2.imread(right_path)

        # Convert to RGB
        original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
        left = cv2.cvtColor(left, cv2.COLOR_BGR2RGB)
        right = cv2.cvtColor(right, cv2.COLOR_BGR2RGB)

        # Show them side by side
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        axs[0].imshow(original)
        axs[0].set_title("Original (Possibly Two Eyes)")
        axs[1].imshow(left)
        axs[1].set_title("Left Eye Split")
        axs[2].imshow(right)
        axs[2].set_title("Right Eye Split")
        for ax in axs:
            ax.axis('off')
        plt.tight_layout()
        plt.show()

        count += 1
        if count == 5:  # limit to 5 previews
            break

In [15]:
print([f for f in os.listdir(split_healthy_path) if "_left" in f or "_right" in f])

[]


In [16]:
import cv2, os

ratios = []
for f in os.listdir('../data/processed/healthy_eye'):
    img = cv2.imread(os.path.join('../data/processed/healthy_eye', f))
    if img is not None:
        h, w = img.shape[:2]
        ratio = w / h
        ratios.append((f, round(ratio, 2)))

# Sort to see widest images first
ratios.sort(key=lambda x: -x[1])
print(ratios[:10])

[('63.jpg', 1.0), ('823.jpg', 1.0), ('189.jpg', 1.0), ('77.jpg', 1.0), ('638.jpg', 1.0), ('604.jpg', 1.0), ('162.jpg', 1.0), ('176.jpg', 1.0), ('88.jpg', 1.0), ('610.jpg', 1.0)]
